In [1]:
pip install -q "tensorflow-text==2.8.*"

Note: you may need to restart the kernel to use updated packages.


In [40]:
#Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from gensim.models import Word2Vec

In [41]:
#Open data
data_file = open('Data.txt').read()
data_file

'The important thing is not to stop questioning. Curiosity has its own reason for existence. One cannot help but be in awe when he contemplates the mysteries of eternity, of life, of the marvelous structure of reality. It is enough if one tries merely to comprehend a little of this mystery each day.'

In [42]:
#Create tokenizer object
tokenizer = Tokenizer()

#Convert data to lowercase
data = data_file.lower().split('.')

In [43]:
#Create dictionary of words with the frequency they occur
#Every word gets unique value > 0
#0 is reserved for padding
tokenizer.fit_on_texts(data)

#Transforms sentences into set of integers from the dictionary
input_sequences = tokenizer.texts_to_sequences(data)

#Counts total words
total_words = len(tokenizer.word_index) + 1

#Prints it all out
print(tokenizer.word_index, '\n')
print(input_sequences, '\n')
print(total_words, '\n')

{'of': 1, 'the': 2, 'is': 3, 'to': 4, 'one': 5, 'important': 6, 'thing': 7, 'not': 8, 'stop': 9, 'questioning': 10, 'curiosity': 11, 'has': 12, 'its': 13, 'own': 14, 'reason': 15, 'for': 16, 'existence': 17, 'cannot': 18, 'help': 19, 'but': 20, 'be': 21, 'in': 22, 'awe': 23, 'when': 24, 'he': 25, 'contemplates': 26, 'mysteries': 27, 'eternity': 28, 'life': 29, 'marvelous': 30, 'structure': 31, 'reality': 32, 'it': 33, 'enough': 34, 'if': 35, 'tries': 36, 'merely': 37, 'comprehend': 38, 'a': 39, 'little': 40, 'this': 41, 'mystery': 42, 'each': 43, 'day': 44} 

[[2, 6, 7, 3, 8, 4, 9, 10], [11, 12, 13, 14, 15, 16, 17], [5, 18, 19, 20, 21, 22, 23, 24, 25, 26, 2, 27, 1, 28, 1, 29, 1, 2, 30, 31, 1, 32], [33, 3, 34, 35, 5, 36, 37, 4, 38, 39, 40, 1, 41, 42, 43, 44], []] 

45 



In [37]:
#Pad sequences
input_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen = 20, padding ='post', value = 0)

In [38]:
input_sequences = np.array(input_sequences)
print(input_sequences.shape)
print(input_sequences)

(5, 20)
[[ 2  6  7  3  8  4  9 10  0  0  0  0  0  0  0  0  0  0  0  0]
 [11 12 13 14 15 16 17  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [19 20 21 22 23 24 25 26  2 27  1 28  1 29  1  2 30 31  1 32]
 [33  3 34 35  5 36 37  4 38 39 40  1 41 42 43 44  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]


In [62]:
#Shift array by one to create targets 
output_sequences = np.array(np.roll(input_sequences, 4))
output_sequences = tf.keras.preprocessing.sequence.pad_sequences(output_sequences, maxlen = 20, padding ='post', value = 0)
output_sequences[-1] = 0
print(output_sequences.shape)
print(output_sequences)

(5, 20)
[[11 12 13 14 15 16 17  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [19 20 21 22 23 24 25 26  2 27  1 28  1 29  1  2 30 31  1 32]
 [33  3 34 35  5 36 37  4 38 39 40  1 41 42 43 44  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]


In [63]:
#Convert to embedding vectors
embedding_model = keras.Sequential()

embedding_model.add(keras.layers.Embedding(20, 512, mask_zero = True, input_length = 20)

embedding_model.compile(loss=keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Nadam(), metrics=[keras.metrics.SparseCategoricalAccuracy()])
                    
keras.utils.plot_model(embedding_model,to_file='model.png', show_shapes=True,expand_nested=True)

history = embedding_model.fit(
  sequences, sequences,
  batch_size=1,
  epochs=10)
word_vectors = embedding_model.predict(sequences)
word_vectors

Epoch 1/10


ValueError: in user code:

    File "/opt/conda/lib/python3.9/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/opt/conda/lib/python3.9/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/conda/lib/python3.9/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/opt/conda/lib/python3.9/site-packages/keras/engine/training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/opt/conda/lib/python3.9/site-packages/keras/engine/training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "/opt/conda/lib/python3.9/site-packages/keras/engine/compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "/opt/conda/lib/python3.9/s

In [ ]:
#Calculate positional encoding
